In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: [Mar 24, 11:59 pm](https://github.com/UBC-CS/cpsc330-2024W2?tab=readme-ov-file#deliverable-due-dates-tentative).**

## Imports

In [2]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from collections import Counter
import re
from sklearn.decomposition import LatentDirichletAllocation
import mglearn


<br><br>

<!-- BEGIN QUESTION -->

<div class="alert alert-info">
    
## Submission instructions
<hr>
rubric={points}

**Please be aware that this homework assignment requires installation of several packages in your course environment. It's possible that you'll encounter installation challenges, which might be frustrating. However, remember that solving these issues is not wasting time but it is an essential skill for anyone aspiring to work in data science or machine learning.**

Follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2024W2/blob/main/docs/homework_instructions.md). 

**You may work in a group on this homework and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 2. 
- Use group work as an opportunity to collaborate and learn new things from each other. 
- Be respectful to each other and make sure you understand all the concepts in the assignment well. 
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. 
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).


When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission.
4. Make sure that the plots and output are rendered properly in your submitted file. 
5. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb. If the pdf or html also fail to render on Gradescope, please create two files for your homework: hw6a.ipynb with Exercise 1 and hw6b.ipynb with Exercises 2 and 3 and submit these two files in your submission.  
</div>

_Points:_ 2

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [3]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [5]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words using `glove_wiki_vectors` embeddings.

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 2

In [6]:
words = ["iphone", "night", "volleyball", "newton"]
words

['iphone', 'night', 'volleyball', 'newton']

In [7]:
glove_wiki_vectors.most_similar(words[0])

[('ipad', 0.9220294952392578),
 ('ipod', 0.8369015455245972),
 ('smartphone', 0.798460841178894),
 ('android', 0.7606974244117737),
 ('app', 0.7491426467895508),
 ('apps', 0.7407809495925903),
 ('ios', 0.7348269820213318),
 ('smartphones', 0.717577338218689),
 ('handsets', 0.7168352007865906),
 ('blackberry', 0.7162630558013916)]

In [8]:
glove_wiki_vectors.most_similar(words[1])

[('evening', 0.8429070115089417),
 ('morning', 0.8382436633110046),
 ('day', 0.8261709809303284),
 ('sunday', 0.8161605000495911),
 ('saturday', 0.8128536343574524),
 ('afternoon', 0.8067009449005127),
 ('weekend', 0.8065499067306519),
 ('days', 0.8000710606575012),
 ('hours', 0.7879374027252197),
 ('week', 0.7607038617134094)]

In [9]:
glove_wiki_vectors.most_similar(words[2])

[('softball', 0.8330720663070679),
 ('badminton', 0.8075590133666992),
 ('handball', 0.7867777943611145),
 ('soccer', 0.7690010666847229),
 ('gymnastics', 0.7622063159942627),
 ('tennis', 0.7554086446762085),
 ('basketball', 0.7434348464012146),
 ('netball', 0.725480318069458),
 ('weightlifting', 0.7158160209655762),
 ('swimming', 0.7148258686065674)]

In [10]:
glove_wiki_vectors.most_similar(words[3])

[('webster', 0.6259992718696594),
 ('hayes', 0.5794194936752319),
 ('carroll', 0.5742024779319763),
 ('cambridge', 0.5738973617553711),
 ('mansfield', 0.5708569288253784),
 ('russell', 0.5666629672050476),
 ('preston', 0.5658804178237915),
 ('fisher', 0.5588769316673279),
 ('compton', 0.5579355359077454),
 ('mason', 0.5537561774253845)]

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [11]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 2

In [12]:
for pair in word_pairs:
    print(
        "The similarity between %s and %s is %0.3f"
        % (pair[0], pair[1], glove_wiki_vectors.similarity(pair[0], pair[1]))
    )

The similarity between coast and shore is 0.700
The similarity between clothes and closet is 0.546
The similarity between old and new is 0.643
The similarity between smart and intelligent is 0.755
The similarity between dog and cat is 0.880
The similarity between tree and lawyer is 0.077


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Representation of all words in English
rubric={points}

**Your tasks:**

1. The vocabulary size of Wikipedia embeddings is quite large. The `test_words` list below contains a few new words (called neologisms) and biomedical domain-specific abbreviations. Write code to check whether `glove_wiki_vectors` have representation for these words or not. 
> If a given word `word` is in the vocabulary, `word in glove_wiki_vectors` will return True. 

In [13]:
test_words = [
    "covididiot",
    "fomo",
    "frenemies",
    "anthropause",
    "photobomb",
    "selfie",
    "pxg",  # Abbreviation for pseudoexfoliative glaucoma
    "pacg",  # Abbreviation for primary angle closure glaucoma
    "cct",  # Abbreviation for central corneal thickness
    "escc",  # Abbreviation for esophageal squamous cell carcinoma
]

<div class="alert alert-warning">

Solution_1_3
    
</div>

_Points:_ 2

In [14]:
for word in test_words:
    if word in glove_wiki_vectors:
        print(f"'{word}' is in the vocabulary. TRUE")
    else:
        print(f"'{word}' is NOT in the vocabulary. FALSE")


'covididiot' is NOT in the vocabulary. FALSE
'fomo' is NOT in the vocabulary. FALSE
'frenemies' is in the vocabulary. TRUE
'anthropause' is NOT in the vocabulary. FALSE
'photobomb' is NOT in the vocabulary. FALSE
'selfie' is NOT in the vocabulary. FALSE
'pxg' is NOT in the vocabulary. FALSE
'pacg' is NOT in the vocabulary. FALSE
'cct' is in the vocabulary. TRUE
'escc' is in the vocabulary. TRUE


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Stereotypes and biases in embeddings
rubric={points}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore this. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

In [15]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

Examples of using analogy to explore biases and stereotypes.  

In [16]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


In [17]:
glove_wiki_vectors.similarity("aboriginal", "success")

0.14283238

In [18]:
glove_wiki_vectors.similarity("white", "success")

0.35182396

<div class="alert alert-warning">

Solution_1_4
    
</div>

_Points:_ 4

In [19]:
glove_wiki_vectors.similarity("asian", "math")

0.16754796

In [20]:
glove_wiki_vectors.similarity("aboriginal", "math")

0.09662896

In [21]:
analogy("asian", "math", "man")

asian : math :: man : ?


,Analogy word,Score
0,teacher,0.611474
1,genius,0.553172
2,guy,0.531245
3,dad,0.528673
4,boy,0.526665
5,kid,0.522451
6,janitor,0.511377
7,doctor,0.509623
8,reading,0.507101
9,taught,0.494365


In [22]:
glove_wiki_vectors.similarity("usa", "football")

0.43687928

In [23]:
glove_wiki_vectors.similarity("china", "football")

0.2688487

In [24]:
glove_wiki_vectors.similarity("woman", "engineer")

0.3340311

In [25]:
glove_wiki_vectors.similarity("man", "engineer")

0.4299851

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Discussion
rubric={points}

**Your tasks:**
1. Discuss your observations from 1.4. Are there any worrisome biases in these embeddings trained on Wikipedia?   
2. Give an example of how using embeddings with biases could cause harm in the real world.

<div class="alert alert-warning">

Solution_1_5
    
</div>

_Points:_ 4

1. Yes, for instance,  the similarity between "Asian" and "math" is much higher (0.1675) compared to the similarity between "Aboriginal" and "math" (0.09663), which may indicate an association with low intelligence for the latter. Moreover, the analogy task "asian : math : man" returned words like “teacher,” “genius,” “guy,” “dad,” and “janitor,” which suggests stereotypical and gendered roles tied to intelligence and occupation. We also saw that "engineer" had a higher similarity score with "man" (0.4299) than with "woman" (0.3340), which reflects gender bias—implying that engineering is more associated with men than with women.
2. Using biased embeddings in real-world applications could cause serious harm. For example, in a résumé screening system, if the embeddings associate "man" more closely with "engineer" and "woman" less so, it might unfairly penalize qualified female applicants for technical roles. Similarly, biased analogies could lead to discriminatory chatbots, recommendation systems, or even judicial risk assessment tools. If unchecked, these biases could perpetuate inequality and undermine fairness in automated systems.

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The goal of topic modeling is discovering high-level themes in a large collection of texts. 

In this homework, you will explore topics in [the 20 newsgroups text dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [26]:
from sklearn.datasets import fetch_20newsgroups

In [27]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [28]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points}

Preprocessing is a crucial step before carrying out topic modeling and it markedly affects topic modeling results. In this exercise, you'll prepare the data using [spaCy](https://spacy.io/) for topic modeling. 

**Your tasks:** 

- Write code using [spaCy](https://spacy.io/) to preprocess the `text` column in the given dataframe `df` and save the processed text in a new column called `text_pp` within the same dataframe.

If you do not have spaCy in your course environment, you'll have to [install it](https://spacy.io/usage) and download the pretrained model en_core_web_md. 

`python -m spacy download en_core_web_md`


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are likely to be more informative for the given task. Some common text preprocessing steps for topic modeling include: 
- getting rid of slashes, new-line characters, or any other non-informative characters
- sentence segmentation and tokenization      
- replacing urls, email addresses, or numbers with generic tokens such as "URL",  "EMAIL", "NUM". 
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization


> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://universaldependencies.org/u/pos/) is the list of part-of-speech tags used in spaCy. 

> You may have to use regex to clean text before passing it to spaCy. Also, you might have to go back and forth between preprocessing in this exercise and and topic modeling in Exercise 2 before finalizing preprocessing steps. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you finalize the preprocessing part, you might want to save the preprocessed data in a CSV and work with this CSV so that you don't run the preprocessing part every time you run the notebook. 
 


In [29]:
import spacy
nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])

<div class="alert alert-warning">

Solution_2_1
    
</div>

_Points:_ 8

In [30]:


def clean_text(text):
    # Replace email addresses
    text = re.sub(r'\b[\w.-]+?@\w+?\.\w+?\b', 'EMAIL', text)
    # Replace URLs
    text = re.sub(r'http\S+|www\.\S+', 'URL', text)
    # Replace numbers
    text = re.sub(r'\b\d+(\.\d+)?\b', 'NUM', text)
    return text

def initial_preprocess(text):
    text = clean_text(text)
    doc = nlp(text)
    tokens = []
    for token in doc:
        if token.is_alpha and not token.is_stop and len(token) > 2:
            tokens.append(token.lemma_.lower())
    return tokens

df["initial_tokens"] = df["text"].apply(initial_preprocess)
all_tokens = [token for tokens in df["initial_tokens"] for token in tokens]
token_freq = Counter(all_tokens)


def final_preprocess(tokens, min_freq=5):
    return " ".join([token for token in tokens if token_freq[token] >= min_freq])
    
# Apply to the dataframe
df["text_pp"] = df["initial_tokens"].apply(final_preprocess)
df["text_pp"]

0       know read num num sence wonder help num num pr...
1       bad question ref list algorithm think bit hard...
2       hand support idea have newsgroup aspect graphi...
3       follow problem bug appreciate help create wind...
4       late upi foreign ministry spokesman tell journ...
                              ...                        
4558    look software call shadow far know simple rayt...
4559    archive faq modified num num num subject num p...
4560    nice answer question difference fed mandate fe...
4561    get try configure specifically need correct xc...
4562    armenians nagarno karabagh simply defend right...
Name: text_pp, Length: 4563, dtype: object

In [31]:
df.iloc[2:6]

,text,target,target_name,initial_tokens,text_pp
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,"[actuallay, hand, support, idea, have, newsgro...",hand support idea have newsgroup aspect graphi...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,"[follow, problem, bug, appreciate, help, creat...",follow problem bug appreciate help create wind...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,"[late, upi, foreign, ministry, spokesman, ferh...",late upi foreign ministry spokesman tell journ...
5,"Hi,\n I'd like to subscribe to Leadership Ma...",5,soc.religion.christian,"[like, subscribe, leadership, magazine, wonder...",like subscribe leadership magazine wonder disk...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification
rubric={points}

**Your tasks:**

- Outline the preprocessing steps you carried out in the previous exercise (bullet point format is fine), providing a brief justification when necessary. 

> You might want to wait to answer this question till you are done with Exercise 2 and you have finalized the preprocessing steps in 2.1. 

<div class="alert alert-warning">

Solution_2_2
    
</div>

_Points:_ 2

- Removed stopwords: These are common words (like "the", "is", "and") that don’t carry meaningful information for topic modeling and could skew topic distribution.
- Lemmatized tokens: Converted words to their base form (e.g., “running” → “run”) to reduce redundancy and group similar words under one root.
- Filtered out short tokens: Removed words with length ≤ 2, which are often noise or irrelevant for topics.
- Excluded non-alphabetic tokens: Dropped punctuation, numbers, and symbols to focus on meaningful language content.
- Converted all tokens to lowercase: Ensures consistency and prevents duplicate versions of the same word (e.g., “Computer” vs. “computer”).
- Replaced email addresses, URLs, and numbers with placeholders like "EMAIL", "URL", "NUM" to anonymize and reduce uniqueness.
- Removed rare or highly unique tokens that aren’t useful for identifying common themes.



<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points}

**Your tasks:**

1. Build LDA models on the preprocessed data using using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) and random state 42. Experiment with a few values for the number of topics (`n_components`). Pick a reasonable number for the number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Points:_ 4

I chose **6 topics** for the LDA model based on the reduced number of categories in the dataset (originally 8), expecting some topics to merge or overlap (e.g., sports categories combining into one). I evaluated different topic numbers, and 6 provided distinct and interpretable groupings of words.

Here are my observations based on the top words per topic:

- **Topic 0**: Appears to capture general **conversation or opinion**, with words like "say", "know", "people", "think", "come", and "day".
- **Topic 1**: Clearly focused on **religion**, with words such as "god", "jesus", "church", "believe", "bible", and "question".
- **Topic 2**: Strongly related to **computing or graphics**, with words like "program", "image", "window", "email", and "available".
- **Topic 3**: Points to **geopolitical/political discussion**, likely on Middle East topics—e.g., "people", "state", "israel", "armenian", "turkish", "government".
- **Topic 4**: Focused on **sports**, as indicated by "game", "year", "team", "player", "season", and "win".
- **Topic 5**: Somewhat less coherent, but likely linked to **technical hardware or scoring systems**, with terms like "num", "period", "power", "van", and "pts" (points?).

Overall, **6 topics** gave a good balance between clarity and specificity. Most topics are clearly aligned with known categories (religion, computing, sports, politics), while others reflect more general themes or technical details.

In [32]:
vec = CountVectorizer(stop_words='english')
X = vec.fit_transform(df["text_pp"])

n_topics = 6
lda = LatentDirichletAllocation(
    n_components=n_topics, learning_method="batch", max_iter=10, random_state=42
)
document_topics = lda.fit_transform(X)
document_topics

array([[8.48749079e-02, 3.01187426e-03, 3.00510781e-03, 8.21665045e-01,
        3.00999414e-03, 8.44330710e-02],
       [2.33896684e-03, 2.33843595e-03, 9.88301990e-01, 2.33370576e-03,
        2.34368023e-03, 2.34322103e-03],
       [1.38027379e-01, 5.76914735e-01, 2.74789307e-01, 3.42813725e-03,
        3.43056927e-03, 3.40987213e-03],
       ...,
       [3.74334730e-03, 1.28714601e-01, 3.72944125e-03, 8.56367867e-01,
        3.73090087e-03, 3.71384283e-03],
       [5.40629455e-03, 5.40483368e-03, 9.72971197e-01, 5.40522195e-03,
        5.40377686e-03, 5.40867601e-03],
       [5.47969126e-04, 5.45606431e-04, 5.45445629e-04, 8.72180686e-01,
        1.25634172e-01, 5.46121559e-04]])

In [33]:
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vec.get_feature_names_out())
mglearn.tools.print_topics(
    topics=range(6),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=10,
)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
say           god           num           num           game          
num           people        file          gun           year          
know          think         program       people        team          
people        believe       use           state         good          
come          know          image         israel        play          
tell          jesus         window        right         think         
think         question      email         armenian      player        
time          thing         available     turkish       win           
day           church        include       government    like          
like          say           run           jews          season        


topic 5       
--------      
num           
period        
pts           
play          
new           
power         
san           
van        

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association
rubric={points}

**Your tasks:**
1. For the number of topics you picked in the previous exercise, show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

_Points:_ 5

In [34]:
# Sort topic components (word importance within each topic)
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vec.get_feature_names_out())

# Print top 10 words per topic
n_words = 10
for topic_idx, topic in enumerate(sorting):
    top_words = feature_names[topic[:n_words]]
    print(f"Topic {topic_idx}: {', '.join(top_words)}")

Topic 0: say, num, know, people, come, tell, think, time, day, like
Topic 1: god, people, think, believe, know, jesus, question, thing, church, say
Topic 2: num, file, program, use, image, window, email, available, include, run
Topic 3: num, gun, people, state, israel, right, armenian, turkish, government, jews
Topic 4: game, year, team, good, play, think, player, win, like, season
Topic 5: num, period, pts, play, new, power, san, van, april, det


In [35]:
topic_labels = {
    0: "General Opinions",
    1: "Religion",
    2: "Computing / Software",
    3: "Geopolitics / Middle East",
    4: "Sports",
    5: "Tech / Stats"
}

for topic_idx, topic in enumerate(sorting):
    top_words = feature_names[topic[:n_words]]
    label = topic_labels.get(topic_idx, "Unknown")
    print(f"Topic {topic_idx} – {label}:\n  {', '.join(top_words)}\n")


Topic 0 – General Opinions:
  say, num, know, people, come, tell, think, time, day, like

Topic 1 – Religion:
  god, people, think, believe, know, jesus, question, thing, church, say

Topic 2 – Computing / Software:
  num, file, program, use, image, window, email, available, include, run

Topic 3 – Geopolitics / Middle East:
  num, gun, people, state, israel, right, armenian, turkish, government, jews

Topic 4 – Sports:
  game, year, team, good, play, think, player, win, like, season

Topic 5 – Tech / Stats:
  num, period, pts, play, new, power, san, van, april, det



<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association
rubric={points}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

_Points:_ 5

In [36]:
# Show the topic distribution for the first 5 documents
import pandas as pd

# document_topics is the output of lda.fit_transform(X)
df_topic_dist = pd.DataFrame(document_topics, columns=[f"Topic_{i}" for i in range(n_topics)])
df_doc_topic = pd.concat([df[["text", "target_name"]], df_topic_dist], axis=1)

# Show first 5 documents with their topic distribution and most likely topic
df_doc_topic["Assigned_Topic"] = df_topic_dist.idxmax(axis=1)
df_doc_topic[["text", "target_name", "Assigned_Topic"] + list(df_topic_dist.columns)].head(5)

,text,target_name,Assigned_Topic,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5
0,"You know, I was reading 18 U.S.C. 922 and some...",talk.politics.guns,Topic_3,0.084875,0.003012,0.003005,0.821665,0.003010,0.084433
1,\n\n\nIt's not a bad question: I don't have an...,comp.graphics,Topic_2,0.002339,0.002338,0.988302,0.002334,0.002344,0.002343
2,"\nActuallay I don't, but on the other hand I d...",comp.graphics,Topic_1,0.138027,0.576915,0.274789,0.003428,0.003431,0.003410
3,"The following problem is really bugging me,\na...",comp.windows.x,Topic_2,0.164963,0.008444,0.801522,0.008372,0.008365,0.008334
4,\n\n This is the latest from UPI \n\n For...,talk.politics.mideast,Topic_3,0.004315,0.004307,0.004301,0.978508,0.004294,0.004275


<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points}

1. Briefly explain how content-based filtering works in the context of recommender systems. 
2. Discuss at least two negative consequences of recommender systems.
3. What is transfer learning in natural language processing? Briefly explain.     

<div class="alert alert-warning">

Solution_3
    
</div>

_Points:_ 6

1. How content-based filtering works in recommender systems:
    - Content-based filtering recommends items to users by analyzing the characteristics (features) of items they have liked in the past and suggesting similar items. For example, if a user watches action movies, the system will recommend other movies with similar genres, directors, or keywords.
2.  Two negative consequences of recommender systems:
    - **Filter bubbles**: Users are repeatedly exposed to similar content, limiting their exposure to diverse viewpoints or options.
    - **Bias reinforcement**: Recommender systems can unintentionally amplify social or cultural biases present in the data, leading to unfair or discriminatory suggestions.
3. What is transfer learning in NLP?
    - Transfer learning in natural language processing involves taking a model trained on a large dataset (e.g., BERT trained on Wikipedia) and fine-tuning it on a smaller, task-specific dataset (e.g., sentiment analysis). This allows the model to leverage previously learned language knowledge and improve performance with less training data.



<!-- END QUESTION -->

<br><br><br><br>

**Before submitting your assignment, please make sure you have followed all the instructions in the Submission instructions section at the top.** 

![](img/eva-well-done.png)